In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

In [ ]:
root_dir = "/gdrive/My Drive/CV/c1"

import sys
sys.path.append(root_dir)

In [ ]:
# 훈련

import os
import cv2
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from skimage.feature import hog
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray
from skimage import img_as_ubyte
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib

# 이미지 경로에서 데이터를 로드합니다.
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        for filename in os.listdir(label_folder):
            img = cv2.imread(os.path.join(label_folder, filename))
            if img is not None:
                images.append(img)
                labels.append(label)
    return images, labels

# 이미지 전처리를 수행합니다.
def preprocess_image(image):
    # Histogram Equalization
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

    # Gaussian Blur
    img_output = cv2.GaussianBlur(img_output, (5, 5), 0)
    
    return img_output


def extract_features(image):
    image_resized = cv2.resize(image, (128, 128))

    # Convert the image to gray scale
    gray = rgb2gray(image)
    
    # Gray Scale Histogram
    hist = np.histogram(gray, bins=256, range=(0.0, 1.0))[0]
    
    # GLCM
    gray = img_as_ubyte(gray)
    glcm = greycomatrix(gray, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)
    contrast = greycoprops(glcm, prop='contrast')[0, 0]
    
    # HOG
    fd, hog_image = hog(gray, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, multichannel=False)
    
    # Ensuring the HOG descriptor size is fixed
    fd = fd[:256]
    
    # Concatenate features into a single numpy array
    feature = np.concatenate([hist, np.array([contrast]), fd], axis=0)
    
    return feature




In [ ]:
 #데이터셋을 로드합니다.
images, labels = load_images_from_folder(root_dir + '/dataset')

# 레이블을 숫자로 변환합니다.
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

# 전처리 및 특징 추출을 수행합니다.
processed_images = [preprocess_image(img) for img in images]
features = [extract_features(img) for img in processed_images]

print(len(features))
print(len(labels))

feature_lengths = [len(feat) for feat in features]
print('Feature lengths:', feature_lengths)

# KFold를 설정합니다.
kf = KFold(n_splits=5, shuffle=True, random_state=42)

model_save_path = '/gdrive/My Drive/CV/c1/models/task2/try1'

# 각 split마다 KNN 모델을 학습하고 평가합니다.
# 각 split마다 KNN 모델을 학습하고 평가합니다.
for fold, (train_index, test_index) in enumerate(kf.split(features)):
    X_train = np.array(features)[train_index]
    X_test = np.array(features)[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

    model = KNeighborsClassifier(n_neighbors=10)
    model.fit(X_train, y_train)

    model_path = root_dir + 'models/task2/try1'
    
    # Ensure the directory exists
    os.makedirs(model_save_path, exist_ok=True)
    
    # Save the model in the specified directory
    joblib.dump(model, f'{model_save_path}/model_{fold}.pkl')

    y_pred = model.predict(X_test)
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

    # Add this line to print the classification report
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [ ]:
# task 2

import os
import cv2
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from skimage.feature import hog
from skimage.feature import greycomatrix, greycoprops
from skimage.color import rgb2gray
from skimage import img_as_ubyte
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib


def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

def preprocess_image(image):
    # Histogram Equalization
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

    # Gaussian Blur
    img_output = cv2.GaussianBlur(img_output, (5, 5), 0)
    
    return img_output


def extract_features(image):
    image_resized = cv2.resize(image, (128, 128))

    gray = rgb2gray(image)
    
    # Gray Scale Histogram
    hist = np.histogram(gray, bins=256, range=(0.0, 1.0))[0]
    
    # GLCM
    gray = img_as_ubyte(gray)
    glcm = greycomatrix(gray, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)
    contrast = greycoprops(glcm, prop='contrast')[0, 0]
    
    # HOG
    fd, hog_image = hog(gray, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True, multichannel=False)
    
    fd = fd[:256]
    
    feature = np.concatenate([hist, np.array([contrast]), fd], axis=0)
    
    return feature


data_dir = root_dir + '/query'
# 데이터셋을 로드합니다.
test_images = load_images_from_folder(data_dir)

processed_images = [preprocess_image(img) for img in test_images]
test_features = [extract_features(img) for img in processed_images]

model_dir = root_dir + '/models/task2/try1/model_0.pkl'

classifier = joblib.load(model_dir)

predict_labels = []
# labels = ['Bicycle', 'Bridge', 'Bus', 'Car', 'Chimney', 'Crosswalk', 'Hydrant', 'Motorcycle', 'Palm', 'Traffic Light']
for feature in test_features:
    neigh_ind = classifier.kneighbors(X=test_features, n_neighbors = 10, return_distance=False)
    neigh_labels = np.array(labels)[neigh_ind]

print(neigh_labels)


In [ ]:
# task2 

import csv

with open(root_dir + '/c1_t2_a1.csv', 'w')as file:
    write = csv.writer(file)
    for i, neigh_label in enumerate(neigh_labels):
        write.writerow([f'query{i+1}.png'] + list(neigh_label))